In [6]:
import pyspark
from pyspark.sql import SparkSession

In [7]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('pyspark_test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/29 11:59:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/29 11:59:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [8]:
!wc -l ../../data/raw/fhvh/fhvhv_tripdata_2021-06.csv

14961893 ../data/raw/fhvh/fhvhv_tripdata_2021-06.csv


In [3]:
df = spark.read \
    .option("header", "true") \
    .csv('../../data/raw/fhvh/fhvhv_tripdata_2021-06.csv')

df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02764|2021-06-01 00:02:41|2021-06-01 00:07:46|         174|          18|      N|                B02764|
|              B02764|2021-06-01 00:16:16|2021-06-01 00:21:14|          32|         254|      N|                B02764|
|              B02764|2021-06-01 00:27:01|2021-06-01 00:42:11|         240|         127|      N|                B02764|
|              B02764|2021-06-01 00:46:08|2021-06-01 00:53:45|         127|         235|      N|                B02764|
|              B02510|2021-06-01 00:45:42|2021-06-01 01:03:33|         144|         146|      N|                  NULL|
|              B02510|2021-06-01 00:18:1

#### pyspark does not infer data types on the columns, like pandas does  
every column is a StringType

In [31]:
df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



Create a dataset with 1000 rows for faster transformations

In [2]:
!head -n 1001 ../../data/raw/fhvh/fhvhv_tripdata_2021-01.csv > ../../data/raw/fhvh/head_fhvhv_tripdata_2021-01.csv

In [3]:
!wc -l ../../data/raw/fhvh/head_fhvhv_tripdata_2021-01.csv

1001 ../../data/raw/fhvh/head_fhvhv_tripdata_2021-01.csv


#### Since the spark dataframe do not infer data types of the columns, you can create a pandas dataframe and then create a spark dataframe from pandas dataframe

In [1]:
import pandas as pd

In [2]:
df_pandas = pd.read_csv('../../data/raw/fhvh/fhvhv_tripdata_2021-06.csv', nrows = 10)

In [3]:
df_pandas.shape

(10, 7)

In [4]:
df_pandas.dtypes

dispatching_base_num      object
pickup_datetime           object
dropoff_datetime          object
PULocationID               int64
DOLocationID               int64
SR_Flag                   object
Affiliated_base_number    object
dtype: object

In [9]:
df = spark.createDataFrame(df_pandas)

df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [10]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

# Initialize SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('pyspark_test') \
    .getOrCreate()

# Read a pandas DataFrame (assuming the path is correct and file exists)
df_pandas = pd.read_csv('../../data/raw/fhvh/fhvhv_tripdata_2021-01.csv', nrows=10)

# Create a PySpark DataFrame from pandas DataFrame
df = spark.createDataFrame(df_pandas)

# Show the DataFrame to verify
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|    NaN|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|    NaN|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|    NaN|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|    NaN|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|    NaN|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

#### Another way to create an appropriate schema for the spark dataframe is to declare the schema manually using pyspark's types module

In [13]:
from pyspark.sql import types

schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
])

# Here the True values are dedicated for the presence of Null values.
# types.StructField function takes in a list

In [4]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('../../data/raw/fhvh/fhvhv_tripdata_2021-01.csv')

df.schema

NameError: name 'spark' is not defined

In [15]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

#### To use all the benefits of Spark engines cluster, the best way is to use partitioned dataframe.

In [16]:
df = df.repartition(24)

# lazy command, only starts when it is saved to another variable

In [17]:
df.write.parquet('../../data/pq/fhvh/2021/01', mode="overwrite")

# Now this code will trigger the partition and write it down to the specified directory
# If you run the code again, you will encounter an error, which states that the folder with the same name already exists
# To overwrite the directory, specifye the mdoe = "mode='overwrite'"

In [18]:
df = spark.read.parquet('../../data/pq/fhvh/2021/01')
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2021-01-03 17:17:21|2021-01-03 17:26:18|         255|          34|   null|
|           HV0003|              B02882|2021-01-05 22:14:07|2021-01-05 22:32:28|         189|         107|   null|
|           HV0003|              B02867|2021-01-02 17:59:55|2021-01-02 18:10:39|          88|         137|   null|
|           HV0003|              B02872|2021-01-02 23:57:54|2021-01-03 00:15:48|         238|         224|   null|
|           HV0003|              B02875|2021-01-06 15:53:13|2021-01-06 16:07:07|         169|         208|   null|
|           HV0003|              B02867|2021-01-07 07:35:24|2021-01-07 07:55:49|

In [26]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('SR_Flag', StringType(), True)])

In [27]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)

